In [ ]:
import json
import sqlite3

# Load JSON data
with open("../data/faq_en.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Connect to SQLite database
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS faq (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    section_name TEXT,
    section_id TEXT,
    faq_subsection_name TEXT,
    questions TEXT,
    response TEXT,
    embeddings BLOB
)
""")

# Parse and insert data
for section in data["data"]:
    section_name = section["attributes"]["Sekcia"]
    section_id = section["attributes"]["SekciaID"]

    for subsection in section["attributes"].get("Podsekcia_s_otazkami_a_odpovedami", []):
        faq_subsection_name = subsection["Podsekcia"]
        
        for qa in subsection["Otazky_a_odpovede"]:
            question = json.dumps([qa["Otazka"]], ensure_ascii=False)
            response = qa["Odpoved"]

            cursor.execute("""
                INSERT INTO faq (section_name, section_id, faq_subsection_name, questions, response)
                VALUES (?, ?, ?, ?, ?)
            """, (section_name, section_id, faq_subsection_name, question, response))

# Commit and close connection
conn.commit()
conn.close()

print("Data inserted successfully!")


In [ ]:
import ast
def generate_questions(question,response, n=5):
    prompt = f"""
    Your task is to generate {n} questions based on the following text:
    Text: "{response}"
    Keep in mind, that the questions should be relevant to the following example question:
    {question}
    Format your questions as a python list, each question should be in double quotes and separated by a comma.
    Example:
    [
        "Where can I donate blood?",
        "Where are the blood donation centres?",
        "Where to go for blood donation?"
    ]
    The questions should be generated based on the context of the provided text. The questions should be open-ended. 
    The questions should be in the language of the provided text.
    """
    response = litellm.completion(
        model="azure/gpt-4",
        deployment_id="gpt-4-turbo",
        messages=[{ "content": prompt,"role": "user"}])
    generated_questions = response.choices[0].message.content
    print(generated_questions)
    return generated_questions


In [ ]:
chitchat_dict={
    "Som Eryc a moje meno je odvodené od slova erytrocyt, čiže červená krvinka. Snažím sa pomôcť ľuďom, aby sa rýchlejšie zorientovali v téme darovania krvi. Okrem toho viem pomôcť aj s plánovaním termínu darovania krvi.":["Ako sa voláš?","Aké je tvoje meno?","Kto si?","Čo dokážeš?","Čo si zač?","Akú máš funkciu?","Akú máš úlohu?","Akú máš prácu?","Čo robíš?","Prečo sa voláš Eryc?","Prečo sa voláš tak ako sa voláš?","Prečo sa voláš Eryc a nie inak?","Prečo sa voláš Eryc a nie Erik?"],
    "I am Eryc and my name is derived from the word erythrocyte, which means red blood cell. I try to help people to get oriented in the topic of blood donation faster. In addition, I can also help with planning a blood donation date.":["What is your name?","What can you do?","What do you call yourself?","Who are you?","What are you?","What is your function?","What is your role?","What is your job?","What do you do?","Why are you called Eryc?","Why are you called what you are called?","Why are you called Eryc and not something else?","Why are you called Eryc and not Eric?"],
}

import sqlite3
import json
import litellm
import os
import numpy as np
# Set up LiteLLM with Azure OpenAI credentials
litellm.api_key = os.getenv("AZURE_API_KEY")
litellm.api_base = os.getenv("AZURE_API_BASE")


def generate_embeddings(query):
    response = litellm.embedding(
        model="azure/text-embedding-3-large",
        deployment_id="text-embedding-3-large",
        api_version="2023-05-15",
        input=query)
    return np.array(response["data"][0]["embedding"], dtype='float32')

# Connect to SQLite
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

for k,v in chitchat_dict.items():

    # Combine questions and response
    combined_text = " ".join(v) + " " + k

    # Get embedding
    embedding_vector = generate_embeddings(combined_text)
    embedding_blob = embedding_vector.tobytes()
    # Store in the database
    cursor.execute("""
        INSERT INTO faq (section_name, section_id, faq_subsection_name, questions, response, embeddings)
        VALUES (?, ?, ?, ?, ?,?)
    """, ("CHITCHAT", "CHITCHAT", "CHITCHAT",json.dumps(v,ensure_ascii=False) , k, embedding_blob))

# Commit and close
conn.commit()
conn.close()

In [ ]:
import json
import sqlite3

# Load JSON data

# Connect to SQLite database
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
ALTER TABLE feedbacks ADD COLUMN TIMESTAMP float;
""")
conn.commit()
conn.close()

In [ ]:

# Connect to SQLite
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()
# Retrieve all rows
cursor.execute("SELECT id, questions, response FROM faq")  # Adjust table name if needed
rows = cursor.fetchall()

for row in rows:
    entry_id, questions_json, response_text = row

    # Convert JSON string to list
    base_question = json.loads(questions_json)[0]
    if entry_id > 36:
        # Generate questions
        generated_questions = json.loads(generate_questions(base_question,response_text)) 
        generated_questions = [question.replace("\n","") for question in generated_questions]
        generated_questions.append(base_question)
        generated_questions = json.dumps(generated_questions, ensure_ascii=False)
        
        # Store in the database
        cursor.execute(
            "UPDATE faq SET questions = ? WHERE id = ?",
            (generated_questions, entry_id)
        )
        conn.commit()
    else:
        continue
# Commit and close

conn.close()

In [ ]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create bookings table
cursor.execute("""
CREATE TABLE IF NOT EXISTS booking (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    donation_type TEXT NOT NULL,
    location TEXT NOT NULL,
    success BOOLEAN NOT NULL,
    status TEXT NOT NULL,
    note TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS slots (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    location TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    slots_total INTEGER NOT NULL,
    slots_remaining INTEGER NOT NULL
)
""")


conn.commit()
conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect("../data/calendar.db")
import random
from datetime import datetime, timedelta

cursor = conn.cursor()

# Locations
locations = [
    "Banská Bystrica", "Bratislava - Kramáre", "Bratislava - Ružinov", "Košice",
    "Martin", "Nitra", "Nové Zámky", "Poprad", "Prešov", "Trenčín", "Trnava", "Žilina"
]

# Date range: March 11 to March 24, 2025
start_date = datetime(2025, 5, 1, 7, 0)  # 07:00 AM on March 11
end_date = datetime(2025, 6, 30, 15, 0)  # 03:00 PM on March 24

# Generate time slots (every 15 minutes from 07:00 to 15:00)
time_slots_per_day = [(start_date + timedelta(minutes=30 * i)).timestamp() // 60 for i in range(0, 8 * 4)] 

# Insert data for each location and each day
current_date = start_date
while current_date <= end_date:
    for location in locations:
        for timestamp in time_slots_per_day:
            adjusted_timestamp = int(timestamp + (current_date - start_date).total_seconds() // 60)
            slots_total = 3
            slots_remaining = random.randint(0, 3)  # Random available slots

            cursor.execute("""
                INSERT INTO slots (location, timestamp, slots_total, slots_remaining)
                VALUES (?, ?, ?, ?)
            """, (location, adjusted_timestamp, slots_total, slots_remaining))

    current_date += timedelta(days=1)  # Move to the next day

# Commit and close connection
conn.commit()
conn.close()

print("Dummy data inserted successfully!")


In [ ]:
def format_timestamp(unix_minutes):
    """
    Converts a Unix timestamp in minutes to a formatted time string.
    
    - If the timestamp is today → returns "HH:MM"
    - If the timestamp is another day → returns "DD.MM. HH:MM"

    :param unix_minutes: Unix timestamp in minutes
    :return: Formatted time string
    """
    dt = datetime.fromtimestamp(unix_minutes * 60)  # Convert to datetime
    today = datetime.now().date()

    if dt.date() == today:
        return dt.strftime("%H:%M")  # Format as "HH:MM"
    else:
        return dt.strftime("%d.%m. %H:%M")  # Format as "DD.MM. HH:MM"

timestamp=29055984

formatted_timestamp = format_timestamp(timestamp)
print(formatted_timestamp)

In [ ]:
from ruamel.yaml import YAML
yaml=YAML()
import json
import io
def yaml_dump(data):
    dump = io.StringIO()
    # yaml.round_trip_dump(data, dump)
    yaml.dump(data, dump)
    return dump.getvalue()
with open("data/donation_questionnaire_responses.json", "r", encoding="utf-8") as file:
    data = json.load(file)
with open("../data/slovak_questionnaire_responses.json") as f:
    slovak_data = json.load(f)

responses_out = {}
for name, response in data.items():
    slovak_text = slovak_data[name]
    responses_out[name] = response
    slot = name.split("utter_ask_")[1]
    responses_out[name].append(dict(text=slovak_text,condition=[dict(type="slot", name="slot_lang", value="sk")],
                                    buttons=[
                                        dict(title="Áno",
                                        payload=f"/SetSlots({slot}=yes)",
                                        ),
                                        dict(title="Nie",
                                            payload=f"/SetSlots({slot}=no)",
                                        )]))
print(yaml_dump(dict(responses=responses_out)))

In [ ]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS forms (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT not null,
    data TEXT,
    timestamp TEXT,
    expires TEXT,
    form_id TEXT
)
""")
conn.commit()
conn.close()

In [ ]:
from datetime import datetime
ts=28941630
def format_timestamp(unix_minutes):
    """
    Converts a Unix timestamp in minutes to a formatted time string.
    
    - If the timestamp is today → returns "HH:MM"
    - If the timestamp is another day → returns "DD.MM. HH:MM"

    :param unix_minutes: Unix timestamp in minutes
    :return: Formatted time string
    """
    dt = datetime.fromtimestamp(unix_minutes * 60)  # Convert to datetime
    today = datetime.now().date()

    if dt.date() == today:
        return dt.strftime("%H:%M")  # Format as "HH:MM"
    else:
        return dt.strftime("%d.%m.%y %H:%M")  # Format as "DD.MM. HH:MM"
print(format_timestamp(ts))

In [ ]:
import sqlite3

conn = sqlite3.connect('../data/faq_database.db')
cursor = conn.cursor()

cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
for name, sql in cursor.fetchall():
    print(f"Table: {name}\n{sql}\n")


In [ ]:
import tiktoken
mytext="""Your task is to analyze the current conversation context and generate a list of actions to start new business processes that we call flows, to extract slots, or respond to small talk and knowledge requests.

These are the flows that can be started, with their description and slots:

flow_donation_questionnaire: Flow to ask all blood donation questionnaire questions. Flow na získanie všetkých otázok z dotazníka pre darovanie krvi.
    slot: slot_auth_agree (Ask the user if they want to authenticate. Spýtaj sa používateľa, či sa chce autentifikovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_wants_to_register (Ask the user if they want to register. Spýtaj sa používateľa, či sa chce zaregistrovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_user_name_and_surname (Ask the user for their name and surname. Spýtaj sa používateľa na jeho meno a priezvisko.)
    slot: slot_user_phone (Ask the user for their phone number. Spýtaj sa používateľa na jeho telefónne číslo.)
    slot: slot_user_address (Ask the user for their address. Spýtaj sa používateľa na jeho adresu.)
    slot: slot_user_dob (Ask the user for their date of birth. Spýtaj sa používateľa na jeho dátum narodenia.)
    slot: slot_user_gender (Ask the user for their gender. Spýtaj sa používateľa na jeho pohlavie)
    slot: slot_access_code (Ask the user for the access code. Spýtaj sa používateľa na prístupový kód.)
    slot: slot_q1
    slot: slot_q2
    slot: slot_q3
    slot: slot_q4
    slot: slot_q5
    slot: slot_q6
    slot: slot_q7
    slot: slot_q8
    slot: slot_q9_1
    slot: slot_q9_2
    slot: slot_q9_3
    slot: slot_q9_4
    slot: slot_q9_5
    slot: slot_q9_6
    slot: slot_q9_7
    slot: slot_q9_8
    slot: slot_q9_9
    slot: slot_q9_10
    slot: slot_q9_11
    slot: slot_q9_12
    slot: slot_q9_13
    slot: slot_q10
    slot: slot_q11
    slot: slot_q12
    slot: slot_q13_1
    slot: slot_q13_2
    slot: slot_q13_3
    slot: slot_q14
    slot: slot_q15
    slot: slot_q16_1
    slot: slot_q16_2
    slot: slot_q16_3
    slot: slot_q17
    slot: slot_q18
    slot: slot_q19_1
    slot: slot_q19_2
    slot: slot_q19_3
    slot: slot_q20
    slot: slot_q21
    slot: slot_q22_1
    slot: slot_q22_2
    slot: slot_q22_3
    slot: slot_q23
    slot: slot_q24
    slot: slot_q25
    slot: slot_q26
    slot: slot_declaration
    
flow_search_faiss: Trigger this flow, if the user asks anything related to blood donation, what is your name or what do you do. Do not start this flow, if the user asks anything related  to the booking a blood donation, cancelling a booking, past blood donations or free time slots. Spusti tento flow, ak sa používateľ spýta na niečo týkajúce sa darovania krvi, ak sa spýta ako sa voláš, alebo čo robíš. Nespúšťaj tento flow, ak sa používateľ spýta na niečo týkajúce sa rezervácie darovania krvi, zrušenia rezervácie, minulých darovaní krvi alebo dostupných termínov pre darovanie krvi.
    
flow_book_donation_slot: Trigger this flow, if the user asks for  blood donation booking. Spusti tento flow, ak sa používateľ spýta na rezerváciu  termínu pre darovanie krvi.
    slot: slot_auth_agree (Ask the user if they want to authenticate. Spýtaj sa používateľa, či sa chce autentifikovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_wants_to_register (Ask the user if they want to register. Spýtaj sa používateľa, či sa chce zaregistrovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_user_name_and_surname (Ask the user for their name and surname. Spýtaj sa používateľa na jeho meno a priezvisko.)
    slot: slot_user_phone (Ask the user for their phone number. Spýtaj sa používateľa na jeho telefónne číslo.)
    slot: slot_user_address (Ask the user for their address. Spýtaj sa používateľa na jeho adresu.)
    slot: slot_user_dob (Ask the user for their date of birth. Spýtaj sa používateľa na jeho dátum narodenia.)
    slot: slot_user_gender (Ask the user for their gender. Spýtaj sa používateľa na jeho pohlavie)
    slot: slot_access_code (Ask the user for the access code. Spýtaj sa používateľa na prístupový kód.)
    slot: slot_rebook (Ask the user if they want to rebook the last booking. Spýtaj sa používateľa, či chce zmeniť poslednú rezerváciu.), allowed values: [True, False]
    slot: slot_auth_agree (Ask the user if they want to authenticate. Spýtaj sa používateľa, či sa chce autentifikovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_wants_to_register (Ask the user if they want to register. Spýtaj sa používateľa, či sa chce zaregistrovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_user_name_and_surname (Ask the user for their name and surname. Spýtaj sa používateľa na jeho meno a priezvisko.)
    slot: slot_user_phone (Ask the user for their phone number. Spýtaj sa používateľa na jeho telefónne číslo.)
    slot: slot_user_address (Ask the user for their address. Spýtaj sa používateľa na jeho adresu.)
    slot: slot_user_dob (Ask the user for their date of birth. Spýtaj sa používateľa na jeho dátum narodenia.)
    slot: slot_user_gender (Ask the user for their gender. Spýtaj sa používateľa na jeho pohlavie)
    slot: slot_access_code (Ask the user for the access code. Spýtaj sa používateľa na prístupový kód.)
    slot: slot_location (Ask the user for the location of the blood donation center. Spýtaj sa používateľa na miesto centra na darovanie krvi.)
    slot: slot_location_choice (Ask the user to select the location of the blood donation center. Spýtaj sa používateľa, aby vybral miesto centra na darovanie krvi.), allowed values: [True, False]
    slot: slot_lookup_date (Ask the user for the date of the blood donation. Try to map the user message to the following values [today, tomorrow, this week, next week] or to the date formats MM-DD or DD.MM. E.g. 13.5. or 13.05. is the same as 05-13. Spýtaj sa používateľa na dátum darovania krvi. Skús namapovať správu používateľa na nasledujúce hodnoty [today, tomorrow, this week, next week] alebo ak je zadaný dátum, na formát MM-DD alebo DD.MM. Napr. 13.5. alebo 13.05. je to isté ako 05-13.)
    slot: slot_booking_time (User specifies the booking time in unix minutes.)
    slot: slot_start_questionnaire (Ask the user if they want to start the questionnaire. Spýtaj sa používateľa, či chce začať s dotazníkom.), allowed values: [True, False]
    slot: slot_auth_agree (Ask the user if they want to authenticate. Spýtaj sa používateľa, či sa chce autentifikovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_wants_to_register (Ask the user if they want to register. Spýtaj sa používateľa, či sa chce zaregistrovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_user_name_and_surname (Ask the user for their name and surname. Spýtaj sa používateľa na jeho meno a priezvisko.)
    slot: slot_user_phone (Ask the user for their phone number. Spýtaj sa používateľa na jeho telefónne číslo.)
    slot: slot_user_address (Ask the user for their address. Spýtaj sa používateľa na jeho adresu.)
    slot: slot_user_dob (Ask the user for their date of birth. Spýtaj sa používateľa na jeho dátum narodenia.)
    slot: slot_user_gender (Ask the user for their gender. Spýtaj sa používateľa na jeho pohlavie)
    slot: slot_access_code (Ask the user for the access code. Spýtaj sa používateľa na prístupový kód.)
    slot: slot_q1
    slot: slot_q2
    slot: slot_q3
    slot: slot_q4
    slot: slot_q5
    slot: slot_q6
    slot: slot_q7
    slot: slot_q8
    slot: slot_q9_1
    slot: slot_q9_2
    slot: slot_q9_3
    slot: slot_q9_4
    slot: slot_q9_5
    slot: slot_q9_6
    slot: slot_q9_7
    slot: slot_q9_8
    slot: slot_q9_9
    slot: slot_q9_10
    slot: slot_q9_11
    slot: slot_q9_12
    slot: slot_q9_13
    slot: slot_q10
    slot: slot_q11
    slot: slot_q12
    slot: slot_q13_1
    slot: slot_q13_2
    slot: slot_q13_3
    slot: slot_q14
    slot: slot_q15
    slot: slot_q16_1
    slot: slot_q16_2
    slot: slot_q16_3
    slot: slot_q17
    slot: slot_q18
    slot: slot_q19_1
    slot: slot_q19_2
    slot: slot_q19_3
    slot: slot_q20
    slot: slot_q21
    slot: slot_q22_1
    slot: slot_q22_2
    slot: slot_q22_3
    slot: slot_q23
    slot: slot_q24
    slot: slot_q25
    slot: slot_q26
    slot: slot_declaration
    
flow_cancel_booking: Trigger this flow, if the user asks for cancelling or rebooking a blood donation booking. Spusti tento flow, ak sa používateľ spýta na zrušenie alebo zmenu rezervácie darovania krvi.
    slot: slot_auth_agree (Ask the user if they want to authenticate. Spýtaj sa používateľa, či sa chce autentifikovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_wants_to_register (Ask the user if they want to register. Spýtaj sa používateľa, či sa chce zaregistrovať.), allowed values: [True, False]
    slot: slot_user_email (Ask the user for their email address. Spýtaj sa používateľa na jeho e-mailovú adresu.)
    slot: slot_user_name_and_surname (Ask the user for their name and surname. Spýtaj sa používateľa na jeho meno a priezvisko.)
    slot: slot_user_phone (Ask the user for their phone number. Spýtaj sa používateľa na jeho telefónne číslo.)
    slot: slot_user_address (Ask the user for their address. Spýtaj sa používateľa na jeho adresu.)
    slot: slot_user_dob (Ask the user for their date of birth. Spýtaj sa používateľa na jeho dátum narodenia.)
    slot: slot_user_gender (Ask the user for their gender. Spýtaj sa používateľa na jeho pohlavie)
    slot: slot_access_code (Ask the user for the access code. Spýtaj sa používateľa na prístupový kód.)
    

===
Here is what happened previously in the conversation:
USER: /SetSlots(slot_q20=no)
AI: Mali ste v priebehu posledných 3 mesiacov pohlavný styk s novým sexuálnym partnerom?
USER: /SetSlots(slot_q21=no)
AI: Boli ste vy alebo váš sexuálny partner pozitívne testovaný na HIV, syfilis alebo infekčnú žltačku?
USER: /SetSlots(slot_q22_1=no)
AI: Boli ste vy alebo váš sexuálny partner užívateľom drog alebo anabolických steroidov?
USER: /SetSlots(slot_q22_2=no)
AI: Boli ste vy alebo váš sexuálny partner poskytovateľom alebo príjemcom sexuálnych služieb za peniaze alebo drogy?
USER: /SetSlots(slot_q22_3=no)
AI: Máte rizikové zamestnanie alebo koníčky? (napr. profesionálny vodič, potápač, práca vo výške)
USER: /SetSlots(slot_q23=no)
AI: Mali ste v posledných 12 mesiacoch (ako muž) sexuálny kontakt s mužom?
USER: /SetSlots(slot_q24=no)
AI: ### Vyhlásenie:  Vyhlasujem, že som porozumel(a) horeuvedeným otázkam a odpovedal(a) som na všetky otázky pravdivo. Som si vedomý(á), že v prípade úmyselného zatajenia akýchkoľvek údajov môžem spôsobiť inému poškodenie zdravia alebo smrť a môžem byť stíhaný(á) podľa Trestného zákona SR. Som oboznámený(á) s možnosťou samovylúčenia a v prípade potreby ju využijem. Považujem sa za vhodného darcu, ktorého krv neohrozí zdravie príjemcu. Ak sa po odbere u mňa objavia známky alebo príznaky infekčného ochorenia, budem o tejto skutočnosti bezodkladne informovať transfúzne pracovisko. Vyhlasujem, že darujem krv (plazmu, krvné bunky) dobrovoľne a bezplatne a súhlasím s jej využitím pre zdravotnícke účely. Vyhlasujem, že som bol(a) zrozumiteľne poučený(á) o účele, povahe a rizikách vyšetrení, ktoré mám ako darca krvi alebo zložiek krvi absolvovať a súhlasím s ich vykonaním.  ### Základná informácia o ochrane osobných údajov:  Vaše osobné údaje uvedené v tomto dotazníku sú spracúvané v informačných systémoch, ktorých prevádzkovateľom je Národná transfúzna služba SR (NTS SR), so sídlom Dúbravská cesta 3/L, 831 01 Bratislava, IČO: 30 853 915, a to:  (a) na účel zabezpečenia kvality, bezpečnosti a činnosti transfúznych liekov,  (b) na účel zabezpečenia kvality a bezpečnosti krvi a zložiek krvi,  (c) na účel spravovania evidencie darcov krvi a zložiek krvi.  Podpisom tohto dotazníka potvrdzujete, že pri odovzdaní dotazníka príslušnému zamestnancovi NTS SR Vám bola poskytnutá základná informácia o ochrane Vašich osobných údajov v informačných systémoch NTS SR.  Bola Vám zároveň poskytnutá možnosť oboznámiť sa s rozšírenou informáciou o ochrane osobných údajov formou:  (a) písomnej informácie na informačnej tabuli v priestoroch NTS SR (ak bol dotazník odovzdaný na pracovisku), alebo  (b) vytlačenej kópie dostupnej u pracovníkov mobilnej odberovej jednotky (ak bol dotazník odovzdaný počas mobilného odberu).  Máte právo na prístup k Vašim osobným údajom, ich opravu a právo na obmedzenie spracúvania, ako je uvedené v rozšírenej informácii o ochrane údajov. Viac informácií nájdete na stránke: [www.ntssr.sk/zasadyochranvudajov](www.ntssr.sk/zasadyochranvudajov)  Otázky o ochrane údajov môžete adresovať na: **osobne.udaje@ntssr.sk** 
USER: /SetSlots(slot_declaration=yes)
AI: Ďakujeme za prijatie vyhlásenia. Dotazník je teraz kompletný. Platnosť dotazníka je 24 hodín.  Ak sa nedostavíte na plánovaný odbor krvi, budete požiadaný(á) o vyplnenie dotazníka s aktuálnymi informáciami.
AI: 
AI: Čím vám ešte môžem pomôcť?
USER: Isiel by si na mojom mieste darovat krv?
AI: Áno, darovanie krvi je veľmi cenný a ušľachtilý čin, ktorý môže pomôcť mnohým ľuďom. Ak sa cítiš dobre a spĺňaš podmienky, určite by som to odporučil!
USER: komu mozem darovat krv ak mam krvnu skupinu 0?

===

You are currently not in any flow and so there are no active slots.
This means you can only set a slot if you first start a flow that requires that slot.

If you start a flow, first start the flow and then optionally fill that flow's slots with information the user provided in their message.

The user just said "komu mozem darovat krv ak mam krvnu skupinu 0?".

===
Based on this information generate a list of actions you want to take. Your job is to start flows and to fill slots where appropriate. Any logic of what happens afterwards is handled by the flow engine. These are your available actions:
* Slot setting, described by "SetSlot(slot_name, slot_value)". An example would be "SetSlot(recipient, Freddy)"
* Starting another flow, described by "StartFlow(flow_name)". An example would be "StartFlow(transfer_money)"
* Cancelling the current flow, described by "CancelFlow()"
* Clarifying which flow should be started. An example would be Clarify(list_contacts, add_contact, remove_contact) if the user just wrote "contacts" and there are multiple potential candidates. It also works with a single flow name to confirm you understood correctly, as in Clarify(transfer_money).
* Intercepting and handle user messages with the intent to bypass the current step in the flow, described by "SkipQuestion()". Examples of user skip phrases are: "Go to the next question", "Ask me something else".
* Responding to knowledge-oriented user messages, described by "SearchAndReply()"
* Responding to a casual, non-task-oriented user message, described by "ChitChat()".
* Handing off to a human, in case the user seems frustrated or explicitly asks to speak to one, described by "HumanHandoff()".


===
Write out the actions you want to take, one per line, in the order they should take place.
Do not fill slots with abstract values or placeholders.
Only use information provided by the user.
Only start a flow if it's completely clear what the user wants. Imagine you were a person reading this message. If it's not 100% clear, clarify the next step.
Don't be overly confident. Take a conservative approach and clarify before proceeding.
If the user asks for two things which seem contradictory, clarify before starting a flow.
If it's not clear whether the user wants to skip the step or to cancel the flow, cancel the flow.
Strictly adhere to the provided action types listed above.
Focus on the last message and take it one step at a time.
Use the previous conversation steps only to aid understanding.

Your action list:"""

# Example: tokenizer for gpt-3.5-turbo
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

tokens = encoding.encode(mytext)

print(f"Token count: {len(tokens)}")


Token count: 4843
